In [ ]:
from __future__ import print_function

import numpy as np
from matplotlib import pyplot as plt
import GPy

`
pip3 install Gpy
pip3 install ipywidgets
`

in case of python 3.7 error while installing


`git clone https://github.com/SheffieldML/GPy
find Gpy -name '*.pyx' -exec cython {} \;
pip3 install Gpy/`

https://github.com/SheffieldML/GPy/issues/649

In [ ]:
from ipywidgets import interact, interactive, widgets, fixed
try:
    from ipywidgets import Layout
except:
    pass

Посмотрим, как зависит от параметров генерация параметров при ядре RBF (Radial basis function)

In [ ]:
@interact(lengthscale=(0.01,1.0), variance=(0.01, 50))
def rbf_kernel_params(lengthscale = 0.5, variance=0.5):
    k = GPy.kern.RBF(1, lengthscale=lengthscale, variance=variance)

    X = np.linspace(0, 5, 500).reshape(-1, 1)

    mu = np.zeros(500)
    C = k.K(X, X)

    Z = np.random.multivariate_normal(mu, C, 3)

    plt.figure()
    for i in range(3):
        plt.plot(X, Z[i, :])
    plt.ylim((-8,8))


Как мы можем создавать ядра?

In [ ]:
def plot_K(k, ax):
    X = np.linspace(-5, 5, 500).reshape(-1, 1)

    mu = np.zeros(500)
    C = k.K(X, X)

    Z = np.random.multivariate_normal(mu, C, 3)

    for i in range(3):
        ax.plot(X, Z[i, :])
    ax.set_title(k.name)

def plot_2(k):
    figure, axes = plt.subplots(2, 1, figsize=(9, 9))
    axes = axes.ravel()
    plot_K(k, axes[1])
    axes[1].set_xlim(left=0, right=5)
    
    k.plot(ax=axes[0])
    axes[0].set_xlim(left=0, right=5)

суммирование ядер

In [ ]:
kerns = ["RBF", "Poly", "StdPeriodic", "Linear", "Bias"]
@interact(kern_1 = kerns, kern_2 = kerns, weight=(0,1.0))
def sum_kernels(kern_1="RBF", kern_2="Poly", weight=1.0):
    if kern_1 == "RBF":
        f_kern = GPy.kern.RBF(1)
    if kern_1 == "Poly":
        f_kern = GPy.kern.Bias(1, 0.01)*GPy.kern.Poly(1)
    if kern_1 == "StdPeriodic":
        f_kern = GPy.kern.StdPeriodic(1)
    if kern_1 == "Linear":
        f_kern = GPy.kern.Linear(1)
    if kern_1 == "Bias":
        f_kern = GPy.kern.Bias(1)
    
    if kern_2 == "RBF":
        s_kern = GPy.kern.RBF(1)
    if kern_2 == "Poly":
        s_kern = GPy.kern.Bias(1, 0.01)*GPy.kern.Poly(1)
    if kern_2 == "StdPeriodic":
        s_kern = GPy.kern.StdPeriodic(1)
    if kern_2 == "Linear":
        s_kern = GPy.kern.Linear(1)
    if kern_2 == "Bias":
        s_kern = GPy.kern.Bias(1)
        
    new_kern = GPy.kern.Bias(1, weight)*f_kern + GPy.kern.Bias(1, 1-weight)*s_kern
    plot_2(new_kern)

произведение ядер

In [ ]:
kerns = ["RBF", "Poly", "StdPeriodic", "Linear", "Bias"]
@interact(kern_1 = kerns, kern_2 = kerns)
def product_kernels(kern_1="RBF", kern_2="Poly"):
    if kern_1 == "RBF":
        f_kern = GPy.kern.RBF(1)
    if kern_1 == "Poly":
        f_kern = GPy.kern.Bias(1, 0.01)*GPy.kern.Poly(1)
    if kern_1 == "StdPeriodic":
        f_kern = GPy.kern.StdPeriodic(1)
    if kern_1 == "Linear":
        f_kern = GPy.kern.Linear(1)
    if kern_1 == "Bias":
        f_kern = GPy.kern.Bias(1)
    
    if kern_2 == "RBF":
        s_kern = GPy.kern.RBF(1)
    if kern_2 == "Poly":
        s_kern = GPy.kern.Bias(1, 0.01)*GPy.kern.Poly(1)
    if kern_2 == "StdPeriodic":
        s_kern = GPy.kern.StdPeriodic(1)
    if kern_2 == "Linear":
        s_kern = GPy.kern.Linear(1)
    if kern_2 == "Bias":
        s_kern = GPy.kern.Bias(1)
        
    new_kern = f_kern*s_kern
    plot_2(new_kern)